Ce script permet de calculer les scores de chaque métier en fonction des critères choisis par l'élève.

In [42]:
### Pour voir les métiers recommandés à un élève, on utilise la fonction show_score(eleve, n_metiers)
### --> eleve : liste des critères choisis par l'élève
### --> n_metiers : nombre de métiers à montrer

import io
import os
import json
import csv
import pandas as pd
import numpy as np

In [47]:
# Préparation des chemins utiles
# Main directory
dir_path = os.getcwd()

# Dossier des données et dossier des arborescences
data_dir_path = os.path.join(dir_path, 'Donnees')
arbo_dir_path = os.path.join(dir_path, "Clean ArborescencesActivitesCompetences")

# Tableau des métiers-variables normalisé:
df_clusters_file = os.path.join(data_dir_path, 'df_clusters.csv')

# Fichier excel de l'arborescences des activités
act_csv = os.path.join(arbo_dir_path,'rome_arboactivités_juin62181.xlsx')
# Fichier excel de l'arborescence des compétences
comp_csv = os.path.join(arbo_dir_path,'rome_arbocompétences_juin62182.xlsx')

# Correspondance Rome-Onisep
rome_onisep_file = os.path.join(data_dir_path, 'Metiers_Onisep_ROME.csv')
rome_onisep = pd.read_csv(rome_onisep_file, sep =',')

# Dossier Output
output_dir_path = os.path.join(dir_path, "Output")

# Lien PE Onisep 
lien_PE_Onisep_path = os.path.join(dir_path, "lien_PE_Onisep.csv")

# Données des métiers
df_clusters = pd.read_csv(df_clusters_file)
matrix_jobs = df_clusters.iloc[:,3:170]
pe_onisep = pd.read_csv(lien_PE_Onisep_path)
pe_onisep = pe_onisep.drop(pe_onisep.columns[[0,2]], axis=1)

In [44]:
pe_onisep.head()

,code_rome,Métier_onisep_1,Métier_onisep_2,Métier_onisep_3,Métier_onisep_4,Métier_onisep_5,Métier_onisep_6,Métier_onisep_7,Métier_onisep_8,Métier_onisep_9,...,Métier_onisep_39,Métier_onisep_40,Métier_onisep_41,Métier_onisep_42,Métier_onisep_43,Métier_onisep_44,Métier_onisep_45,Métier_onisep_46,Métier_onisep_47,Métier_onisep_48
0,A1101,conducteur / conductrice de machines agricoles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A1201,technicien forestier / technicienne forestière,ouvrier forestier / ouvrière forestière,élagueur / élagueuse,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A1202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A1203,ouvrier / ouvrière paysagiste,technicien / technicienne paysagiste,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1204,garde à cheval,"garde (chasse, pêche, littoral, rivière, parcs...",chargé / chargée d'études naturalistes,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Elèves fictifs avec les affinités qu'on pourrait imaginer(technicien, manager, artiste)
# On peut rajouter l'élève obtenu avec le POC "Choix des critères par l'utilisateur"
brice = [#'Jouer de la musique ou chanter',
 #'Être acteur ou comédien, danser ou défiler',
 #'Analyser des données techniques',
 'Corriger des textes, rédiger, traduire ou collecter des informations',
 #'Connaissances artistiques',
 #'Connaissances linguistiques',
 #'Connaissances littéraires',
 #'Compétences artistiques',
 "Compétences d'écriture",
 #'Compétences mentales',
 #'Sciences physico-chimiques',
 #'Sciences géologiques',
 #'Sciences humaines et sociales',
 #'Techniques de communication'
]

karine1 = [
    'Cultiver ou entretenir des espaces naturels ',
    'Réaliser un art plastique',
    'Jouer de la musique ou chanter',
    'Prendre soin de personnes',
    'Corriger des textes, rédiger, traduire ou collecter des informations',
    'Connaissances du bois',
    'Soins aux personnes',
    'Connaissances artistiques',
    'Connaissances linguistiques',
    'Connaissances littéraires',
    'Compétences artistiques',
    'Compétences d’écriture',
    'Compétences gestuelles',
    'Appréciation sensorielle',
    'Interprétation de données ou de signaux',
    'Compétences mentales',
    'Compétences physiques',
    'Sciences mathématiques',
    'Sciences physico-chimiques',
    'Sciences du vivant',
    'Sciences humaines et sociales',
    'Sciences médicales'
]

karine2 = [
    #'Cultiver ou entretenir des espaces naturels ',
    'Réaliser un art plastique',
    'Jouer de la musique ou chanter',
    'Prendre soin de personnes',
    'Corriger des textes, rédiger, traduire ou collecter des informations',
    #'Connaissances du bois',
    'Soins aux personnes',
    'Connaissances artistiques',
    'Connaissances linguistiques',
    'Connaissances littéraires',
    'Compétences artistiques',
    'Compétences d’écriture',
    'Compétences gestuelles',
    'Appréciation sensorielle',
    'Interprétation de données ou de signaux',
    'Compétences mentales',
    'Compétences physiques',
    #'Sciences mathématiques',
    #'Sciences physico-chimiques',
    'Sciences du vivant',
    'Sciences humaines et sociales',
    'Sciences médicales'
]


kevin = ['Analyser des données socio-économiques',
 'Prendre des décisions et définir une stratégie',
 'Coordonner un projet, des actions ou une formation',
 'Gérer une production industrielle',
 'Communiquer',
 'Pratiquer une activité sportive',
 'Vendre des produits ou des services',
 'Acheter ou négocier des produits ou des services',
 'Compétences artistiques',
 'Compétences d’écriture',
 'Compétences sportives',
 'Compétences managériales',
 'Connaissances linguistiques',
 'Sciences économiques']

leonore = ['Conseiller en tant qu’expert un client',
 'Analyser et déterminer les besoins d’un client',
 'Assurer une étape manuelle de production industrielle (assembler, monter, souder, usiner, mouler, imprimer, traiter, …)',
 'Gérer le budget ou la comptabilité d’une structure',
 'Planifier',
 'Gérer un dossier ou une formalité administrative',
 'Saisir et mettre en forme des documents',
 'Compléter des documents ou des dossiers',
 'Organiser un événement ou superviser une opération',
 'Enseigner',
 'Analyser des données techniques',
 'Enquêter',
 'Concevoir une offre, un produit ou un processus',
 'Corriger des textes, rédiger, traduire ou collecter des informations',
 'Communiquer',
 'Traiter des images, des sons, des lumières, des contenus multimédias',
 'Gérer une entreprise ou un service',
 'Prendre des décisions et définir une stratégie',
 'Superviser une équipe',
 'Coordonner un projet, des actions ou une formation',
 'Décorer ou agencer un espace',
 'Être acteur ou comédien, danser ou défiler',
 'Jouer de la musique ou chanter',
 'Concevoir un projet artistique',
 'Sciences mathématiques',
 'Sciences chimiques',
 'Sciences physiques',
 'Sciences physico-chimiques',
 'Compétences artistiques',
 'Compétences d’écriture',
 'Compétences techniques en lumière',
 'Compétences techniques en son',
 'Connaissances artistiques',
 'Connaissances linguistiques',
 'Compétences mécaniques',
 'Techniques de procédés physiques ou chimiques',
 'Techniques industrielles d’assemblage et de traitement',
 'Techniques industrielles artisanales et manuelles',
 'Techniques de métallurgie, soudure et usinage',
 'Techniques de menuiserie',
 "Méthodes d'ingénierie",
 'Procédures techniques industrielles',
 'Appréciation sensorielle']


eleve = ['Compétences sportives', 'Compétences multimédia', 'Planifier', 'Gérer un approvisionnement et des stocks', 'Gérer une production industrielle']

marie = ['Animer un groupe',
 'Enseigner',
 'Prendre soin de personnes',
 'Gérer un dossier ou une formalité administrative',
 'Compléter des documents ou des dossiers',
 'Saisir et mettre en forme des documents',
 'Ressources humaines',
 'Compétences artistiques',
 'Interprétation de données ou de signaux',
 'Compétences managériales',
 'Techniques de prévention et gestion de conflits',
 'Techniques pédagogiques',
 'Compétences mentales']
## Fonctions utilisées

In [52]:
#### change le nom des variables des élèves pour les faire correspondre avec ceux de R
def eleve_cast_R(eleve_to_transform):
    eleve = []
    for feature in eleve_to_transform:
        feature = feature.replace(', ','..')
        feature = feature.replace('(', '.')
        feature = feature.replace(')', '.')
        feature = feature.replace("'", '.')
        feature = feature.replace("/", '.')
        feature = feature.replace(' ', '.')
        eleve.append(feature)
    return(eleve)

### Transforme le résultat du questionnaire en vecteur élève
# Chargement des données de compétences dans la variable comp
comp = pd.read_excel(comp_csv, sheetname="Arbo Comptétences ROMEJuin 2016", header=0)
act = pd.read_excel(act_csv, sheetname='Arbo Activités ROME Juin 2016', header=0)

def cast_eleve(eleve_to_cast):
    eleve_cast = []
    for item in eleve_to_cast:
        item_list = [item]
        # L'intitulé 'Techniques de plongée' a été retiré du questionnaire, et il est associé ici à 
        # 'Compétences sportives" : si un élève choisit cette dernière compétence, on lui associera
        # automatiquement la compétence 'Techniques de plongée'
        if item=='Compétences sportives':
            item_list.append("Techniques de plongée")
        for element in item_list:    
            try:
                index = comp[comp["Libellé questionnaire"]==element]['Niveau 2'].index.tolist()[0]
                if str(comp.iloc[index,1])!='nan':
                    eleve_cast.append(str(comp.iloc[index,1]))
                elif str(comp.iloc[index,2])!='nan':
                    eleve_cast.append(str(comp.iloc[index,2]))
            except:
                pass
            
        for element in item_list:        
            try:
                index = act[act["Libellé questionnaire"]==element]['Niveau 2'].index.tolist()[0]
                if str(act.iloc[index,1])!='nan':
                    eleve_cast.append(str(act.iloc[index,1]))
                elif str(act.iloc[index,2])!='nan':
                    eleve_cast.append(str(act.iloc[index,2]))
            except:
                pass            
    return eleve_cast

#### vecteur élève (petit problème à régler encore)
def vectorize_eleve(eleve_to_vectorize): 
    eleve = pd.DataFrame()
    for var in matrix_jobs.columns.tolist():
        if var in eleve_to_vectorize:  
            eleve[var]=[1]
        else:
            eleve[var]=[0]
    return eleve

#### Score des produits scalaires de chaque métier
intitule = df_clusters["intitule"]
clusters = df_clusters["clusters"]
rome = df_clusters["code_rome"]
X1 = df_clusters["X1"]
X2 = df_clusters["X2"]
cluster = df_clusters["clusters"]

def score_jobs(eleve_to_match):
    eleve_c = cast_eleve(eleve_to_match)
    eleve_R = eleve_cast_R(eleve_c)
    eleve_v = vectorize_eleve(eleve_R)
    score = matrix_jobs.dot(eleve_v.transpose())
    df = pd.concat([intitule,rome, score, X1, X2, cluster], axis=1)
    df.columns = ["intitule","code_rome","score","X1","X2","cluster"]
    df = pd.merge(df, pe_onisep, on="code_rome")
    return(df.sort_values(by="score",ascending=False))

def show_score(eleve, n_metiers):
    score_eleve = score_jobs(eleve)
    print("--> Préférences de l'élève : \n")
    for item in eleve:
        print(item)
    print("\n --> Métiers recommandés :")
    print(score_eleve[0:n_metiers])
    
def output_score(eleve, name):
    output_path = os.path.join(output_dir_path, name + ".csv")
    score_eleve = score_jobs(eleve)
    score_eleve.to_csv(output_path)
    

In [53]:
output_score(brice, "Brice")

In [5]:
show_score(brice, 20)

--> Préférences de l'élève : 

Corriger des textes, rédiger, traduire ou collecter des informations
Compétences d'écriture

 --> Métiers recommandés :
                                              intitule   rome      score
130     Préparation et correction en édition et presse  E1305  15.836394
118                   Journalisme et information média  E1106  10.126388
120                         Traduction, interprétariat  E1108   8.984387
384    Gestion de l'information et de la documentation  K1601   6.700385
114                     Ecriture d'ouvrages, de livres  E1102   3.845382
115                                      Communication  E1103   3.274382
113                       Animation de site multimédia  E1101   0.990379
117                             Coordination d'édition  E1105   0.990379
142                             Mesures topographiques  F1107   0.990379
375                          Conseil en Santé Publique  K1402   0.990379
490  Exploitation de systèmes de communication

In [ ]:
show_score(karine2, 20)

In [ ]:
show_score(karine1, 30)

In [ ]:
show_score(kevin, 40)

In [ ]:
show_score(leonore, 40)

In [ ]:
show_score(marie, 20)

In [ ]:
show_score(eleve, 20)

In [ ]:
rome_final = pd.DataFrame(columns=["intitule_onisep","rome"])
for rome in df_clusters["code_rome"]:
    if rome in list(rome_onisep['code_ROME (Impala)']):
        index = rome_onisep[rome_onisep["code_ROME (Impala)"]==rome].index.tolist()[0]
        oni = rome_onisep.iloc[index,1]

output_file = os.path.join(dir_path, 'rome_manquant.csv')
PE_manquant.to_csv(output_file)